In [26]:
from hazm import *
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from tqdm import tqdm
import itertools
import random
import pickle
from sklearn.model_selection import train_test_split
from taggers import PerceptronTagger
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

In [2]:
with open('/Users/e_ghafour/repos/pos_tagger/data.pkl', 'rb') as ff:
    data = pickle.load(ff)

In [3]:
words = []
tags = []
for sent in tqdm(data):
    word_tmp = []
    tags_tmp = []
    for word, tag in sent:
        word_tmp.append(word)
        tags_tmp.append(tag)
    words.append(word_tmp)
    tags.append(tags_tmp)

100%|██████████| 344741/344741 [00:04<00:00, 78411.16it/s] 


In [4]:
X_train, X_test, y_train, y_test = train_test_split(words, tags, test_size=0.1, random_state=10)

In [5]:
sentences = list(zip(X_train, y_train))

In [6]:
classes = list(np.unique(list(itertools.chain(*y_train))))

In [7]:
perceptron = PerceptronTagger(load=False)

In [8]:
perceptron.train(sentences=sentences,
                 save_loc='here.pkl')

0


100%|██████████| 310266/310266 [03:20<00:00, 1549.84it/s]


1


100%|██████████| 310266/310266 [03:25<00:00, 1506.99it/s]


2


100%|██████████| 310266/310266 [03:25<00:00, 1509.53it/s]


3


100%|██████████| 310266/310266 [03:32<00:00, 1460.95it/s]


4


100%|██████████| 310266/310266 [03:29<00:00, 1482.65it/s]


In [24]:
def my_tag(model, tokenize_sents):
    tokens = []
    prev, prev2 = model.START
    for words in tqdm(tokenize_sents):
        token_sent = []
        context = model.START + [model._normalize(w) for w in words] + model.END
        for i, word in enumerate(words):
            tag = model.tagdict.get(word)
            if not tag:
                features = model._get_features(i, word, context, prev, prev2)
                tag = model.model.predict(features)
            token_sent.append(tag)
            prev2 = prev
            prev = tag
        tokens.append(token_sent)
    return tokens

In [25]:
y_pred = my_tag(perceptron, X_test)


100%|██████████| 34475/34475 [00:21<00:00, 1597.67it/s]


In [28]:
y_pred_flatten = list(itertools.chain(*y_pred))
y_test_flatten = list(itertools.chain(*y_test))

In [30]:
print(classification_report(y_pred=y_pred_flatten , y_true=y_test_flatten))

print('Precision                                   : %.5f'%precision_score(y_pred=y_pred_flatten , y_true=y_test_flatten, average='weighted'))
print('Recall                                      : %.5f'%recall_score(y_pred=y_pred_flatten , y_true=y_test_flatten, average='weighted'))
print('F1-Score                                    : %.5f'%f1_score(y_pred=y_pred_flatten , y_true=y_test_flatten, average='weighted'))


              precision    recall  f1-score   support

         ADJ       0.94      0.94      0.94     67838
      ADJ,EZ       0.91      0.91      0.91     22917
         ADP       1.00      1.00      1.00    113104
      ADP,EZ       0.98      0.99      0.98     11121
         ADV       0.95      0.93      0.94     18021
      ADV,EZ       0.97      0.92      0.94      1125
       CCONJ       1.00      1.00      1.00     59224
    CCONJ,EZ       1.00      0.99      0.99        99
         DET       0.97      0.98      0.97     22189
      DET,EZ       0.92      0.94      0.93      2336
        INTJ       0.89      0.97      0.93        67
        NOUN       0.96      0.95      0.96    211864
     NOUN,EZ       0.95      0.97      0.96    180460
         NUM       0.98      0.99      0.99     26462
      NUM,EZ       0.90      0.83      0.86      1911
        PRON       0.97      0.97      0.97     25418
     PRON,EZ       0.84      0.67      0.74       275
       PUNCT       1.00    